<a href="https://colab.research.google.com/github/inesgh1/scripted-Chatbot/blob/main/mychatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import pandas as pd
import numpy as np
import tensorflow
import nltk
nltk.download('punkt')
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Loading our JSON Data**

In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy

import tensorflow
import random

import json
with open('intents.json') as file:
    data = json.load(file)

**Extracting Data**

In [3]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

In [4]:
for intent in data['intents']:
    for pattern in intent['patterns']:
  # For each pattern I will turn it into a list of words using nltk.word_tokenizer
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [5]:
#Stemming a word is attempting to find the root of the word.
#perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

**Bag of Words**

represent each sentence with a list the length of the amount of words in the models vocabulary.
Each position in the list will represent a word from the vocabulary. If the position in the list is a 1 then that will mean that the word exists in the sentence, if it is a 0 then the word is nor present. 

In [6]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
  #initialize bag of words
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)
#convert the training data and output to numpy arrays.    
training = numpy.array(training)
output = numpy.array(output)    
with open("data.pickle", "wb") as f:
      pickle.dump((words, labels, training, output), f)

**Developing a Model**

using feed-forward neural network with two hidden layers. The goal of the network will be to look at a bag of words and give a class that they belong too (one of the tags from the JSON file).

In [7]:
!pip install tflearn

     |████████████████████████████████| 107 kB 5.4 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=c5b9e5e85e0c11fdf5cdecd1c66585d3b2f119b072a58168e9f1ce64a0d1af73
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [10]:
# defining the architecture of our model
try:
    model.load('model.tflearn')
except:
    tensorflow.compat.v1.reset_default_graph() 
    import tflearn

    net = tflearn.input_data(shape=[None, len(training[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
    net = tflearn.regression(net)

    model = tflearn.DNN(net)

    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")


Training Step: 3999  | total loss: 0.56571 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.56571 - acc: 0.9502 -- iter: 24/26
Training Step: 4000  | total loss: 0.51776 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.51776 - acc: 0.9552 -- iter: 26/26
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


**Training & Saving the Model**

In [15]:
#The bag_of_words function will transform the string input to a bag of words using the created words list
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

#The chat function will handle getting a prediction from the model and grabbing an appropriate 
#response from our JSON file of responses
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hi
Hi there, how can I help?
You: what hours are you open!!
Our hours are 9am-9pm every day
You: are you open today!!
Our hours are 9am-9pm every day
You: do you take credit cards!!
We accept VISA, Mastercard and AMEX
You: thanks
Any time!
You: quit
